In [ ]:
'''
Author: Graham MacGregor (Graham.macgregor@gov.bc.ca) 
-Created from some examples on the web
Date: April 2021
Purpose: To Update a AGOL feature service with a feature layer from a filegeodatabase
Python Version: 3
Assumptions:
-Editable ArcGIS account
-Update of service is exact same data schema otherwise applications linked to it will break

History:

'''

In [ ]:
#Import python libraries
import arcpy, os, time
from zipfile import ZipFile
from arcgis.gis import GIS

In [ ]:
#Set overwrite and clock to track time
arcpy.env.overwriteOutput = True
startTime = time.clock()

In [ ]:
# Variables. 
username = "GSMACGRE.BC"                                    # AGOL Username
password = "******"                                         # AGOL Password
fc = r"W:\FOR\RSI\DQU\Projects\2021\Minor_Projects\hackathon\data\MLIGA.gdb\MLIGA_POLY"  #Feature class in filegeodatabase                           # Path to Feature Class
fsItemId = "2868935c37f94a6ab5ab5c7c0213b2d4" #Feature Service Item ID

In [ ]:
#Log in with your credentials
gis = GIS("https://www.arcgis.com", username, password) # Log into ArcGIS

In [ ]:
# Function to Zip FGDB. Used to zip temporary filegeodatabase that will be used to update feature service
def zipDir(dirPath, zipPath):
    zipf = ZipFile(zipPath , mode='w')
    gdb = os.path.basename(dirPath)
    for root, _ , files in os.walk(dirPath):
        for file in files:
            if 'lock' not in file:
               filePath = os.path.join(root, file)
               zipf.write(filePath , os.path.join(gdb, file))
    zipf.close()

In [ ]:
#Create a temporary filegeodatabase for zipping up fc feature
print("Creating temporary File Geodatabase")
arcpy.CreateFileGDB_management(arcpy.env.scratchFolder, "TempGDB")

In [ ]:
# Export feature classes to the temporary File Geodatabase
fcName = os.path.basename(fc)
fcName = fcName.split('.')[-1]
print("Exporting {0} to TempGDB".format(fcName))
arcpy.conversion.FeatureClassToFeatureClass(fc, os.path.join(arcpy.env.scratchFolder, "TempGDB.gdb"), fcName)

In [ ]:
# Zip temp FGDB
print("Zipping TempGDB")
zipDir(os.path.join(arcpy.env.scratchFolder, "TempGDB.gdb"), os.path.join(arcpy.env.scratchFolder, "TempGDB.gdb.zip"))

In [ ]:
# Check if FGDB exists in the AGOL users environment, if True, delete item
searchResults = gis.content.search('title:TempGDB AND owner:{0}'.format(username), item_type='File Geodatabase')
if len(searchResults) > 0:
    item = searchResults[0]
    item.delete()

In [ ]:
# Upload zipped File Geodatabase with set properties
print("Uploading File Geodatabase")
fgd_properties={'title':'tempGDB', 'tags':'temp file geodatabase', 'type':'File Geodatabase'}
fgd_item = gis.content.add(item_properties=fgd_properties, data=os.path.join(arcpy.env.scratchFolder, "TempGDB.gdb.zip"))

In [ ]:
# Truncate Feature Service
print("Truncating Feature Service")
premiseLayer = gis.content.get(fsItemId)
fLyr = premiseLayer.layers[0]
fLyr.manager.truncate()

# Append features from feature class in zipped file geodatabase
print("Appending features")
fLyr.append(item_id=fgd_item.id, upload_format="filegdb", upsert=False, field_mappings=[])

# Delete Uploaded File Geodatabase
print("Deleting uploaded File Geodatabase")
fgd_item.delete()

In [ ]:
# Delete temporary File Geodatabase and zip file
print("Deleting temporary FGD and zip file")
arcpy.Delete_management(os.path.join(arcpy.env.scratchFolder, "TempGDB.gdb"))
os.remove(os.path.join(arcpy.env.scratchFolder, "TempGDB.gdb.zip"))

#the time it took to run the script
endTime = time.clock()
elapsedTime = round((endTime - startTime) / 60, 2)
print("Script finished in {0} minutes".format(elapsedTime))